In [20]:
import os
import errno
import pickle
import requests
import xmltodict, json
import webbrowser
import datetime
from time import time, strftime, localtime
from datetime import timedelta
import urllib
import pandas as pd
import sqlite3

In [84]:
curr_path = os.getcwd()
sub_path = curr_path +"/Dataset/Trade"

In [85]:
sub_path

'/Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade'

##  구 코드 db 생성

#### get gu_code, name method

In [4]:
SI_CODE = 0

In [28]:
kb_db = sqlite3.connect("../KB/KB.db")

In [34]:
def getAddrInfo(code, type='SGG'):
    global SI_CODE
    SI_CODE = code
    response = urllib.request.urlopen("http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?bjd_code=%d&bjd_gbn=%s" %(code,type) )
    data = json.loads(response.read().decode('utf-8'))
    data = pd.DataFrame(data[u'resultList'])
    return data

gu_df = getAddrInfo(11)
gu_df['SI'] = (SI_CODE)
gu_df = gu_df[['SI', 'CODE', 'CODE_VALUE']]
gu_df['CODE'] = gu_df['CODE'].astype('int')
gu_df.head(10)


,SI,CODE,CODE_VALUE
0,11,680,강남구
1,11,740,강동구
2,11,305,강북구
3,11,500,강서구
4,11,620,관악구
5,11,215,광진구
6,11,530,구로구
7,11,545,금천구
8,11,350,노원구
9,11,320,도봉구


In [24]:
gu_info = getAddrInfo(11)

In [25]:
values= gu_info.apply(lambda x : u"(%d, %s, '%s')" % (11, x['CODE'], x['CODE_VALUE'] ), axis=1)
values

0      (11, 680, '강남구')
1      (11, 740, '강동구')
2      (11, 305, '강북구')
3      (11, 500, '강서구')
4      (11, 620, '관악구')
5      (11, 215, '광진구')
6      (11, 530, '구로구')
7      (11, 545, '금천구')
8      (11, 350, '노원구')
9      (11, 320, '도봉구')
10    (11, 230, '동대문구')
11     (11, 590, '동작구')
12     (11, 440, '마포구')
13    (11, 410, '서대문구')
14     (11, 650, '서초구')
15     (11, 200, '성동구')
16     (11, 290, '성북구')
17     (11, 710, '송파구')
18     (11, 470, '양천구')
19    (11, 560, '영등포구')
20     (11, 170, '용산구')
21     (11, 380, '은평구')
22     (11, 110, '종로구')
23      (11, 140, '중구')
24     (11, 260, '중랑구')
dtype: object

In [ ]:
insert_sql = u"insert into addr_gu_code_tbl( si_code, gu_code, name) values %s ;" % (u",".join(values))
kb_db.execute(insert_sql)
kb_db.commit()

In [19]:
for si,gu in gu_df[['SI','CODE']].itertuples(index=False):
    code = "%d%d" % (si,gu)
    info = getAddrInfo(int(code),'EMD')
    print(info)
#     values=info.apply(lambda x : u"(%d, %d, %s, '%s')" % (si,gu, x['CODE'], x['CODE_VALUE'] ), axis=1)
#     insert_sql = u"insert into addr_dong_code_tbl( si_code, gu_code, dong_code,name) values %s ;" % (u",".join(values))
#     kb_db.execute(insert_sql)
#     kb_db.commit()
#     print ("%s code inserted(%d)" % (code, info.count()[0]))

   CODE CODE_VALUE
0   103        개포동
1   108        논현동
2   106        대치동
3   118        도곡동
4   105        삼성동
5   111        세곡동
6   115        수서동
7   107        신사동
8   110       압구정동
9   101        역삼동
10  113        율현동
11  114        일원동
12  112        자곡동
13  104        청담동
  CODE CODE_VALUE
0  110        강일동
1  102        고덕동
2  105         길동
3  106        둔촌동
4  101        명일동
5  103        상일동
6  108        성내동
7  107        암사동
8  109        천호동
  CODE CODE_VALUE
0  101        미아동
1  102         번동
2  103        수유동
3  104        우이동
   CODE CODE_VALUE
0   104        가양동
1   110        개화동
2   108        공항동
3   111        과해동
4   106       내발산동
5   102        등촌동
6   105        마곡동
7   109        방화동
8   101        염창동
9   112        오곡동
10  113        오쇠동
11  107       외발산동
12  103        화곡동
  CODE CODE_VALUE
0  103        남현동
1  101        봉천동
2  102        신림동
  CODE CODE_VALUE
0  104        광장동
1  103        구의동
2  109        군자동
3  102         능동
4  105        자양동

In [33]:
town_info=pd.read_sql("select * from addr_dong_code_tbl;",kb_db)
town_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 4 columns):
si_code      467 non-null int64
gu_code      467 non-null int64
dong_code    467 non-null int64
name         467 non-null object
dtypes: int64(3), object(1)
memory usage: 14.7+ KB


In [48]:
gu_code_df = town_info.merge(gu_df, left_on=['si_code', 'gu_code'], right_on=['SI', 'CODE'], how='inner')
gu_code_df['si_name'] = '서울시'
gu_code_df = gu_code_df.drop(['si_code', 'gu_code'], 1)
gu_code_df

,dong_code,name,SI,CODE,CODE_VALUE,si_name
0,103,개포동,11,680,강남구,서울시
1,108,논현동,11,680,강남구,서울시
2,106,대치동,11,680,강남구,서울시
3,118,도곡동,11,680,강남구,서울시
4,105,삼성동,11,680,강남구,서울시
5,111,세곡동,11,680,강남구,서울시
6,115,수서동,11,680,강남구,서울시
7,107,신사동,11,680,강남구,서울시
8,110,압구정동,11,680,강남구,서울시
9,101,역삼동,11,680,강남구,서울시


### dataframe column re-order

In [49]:
gu_code_df = gu_code_df[['SI', 'CODE', 'dong_code', 'si_name','CODE_VALUE', 'name']]
gu_code_df

,SI,CODE,dong_code,si_name,CODE_VALUE,name
0,11,680,103,서울시,강남구,개포동
1,11,680,108,서울시,강남구,논현동
2,11,680,106,서울시,강남구,대치동
3,11,680,118,서울시,강남구,도곡동
4,11,680,105,서울시,강남구,삼성동
5,11,680,111,서울시,강남구,세곡동
6,11,680,115,서울시,강남구,수서동
7,11,680,107,서울시,강남구,신사동
8,11,680,110,서울시,강남구,압구정동
9,11,680,101,서울시,강남구,역삼동


### dataframe column re-name

In [50]:
gu_code_df.columns = ['si_code', 'gu_code', 'dong_code', 'si_name', 'gu_name', 'dong_name']

In [53]:
gu_code_df.head()

,si_code,gu_code,dong_code,si_name,gu_name,dong_name
0,11,680,103,서울시,강남구,개포동
1,11,680,108,서울시,강남구,논현동
2,11,680,106,서울시,강남구,대치동
3,11,680,118,서울시,강남구,도곡동
4,11,680,105,서울시,강남구,삼성동


In [77]:
tmp = pd.DataFrame(gu_code_df['si_code'].astype(str) + gu_code_df['gu_code'].astype(str))
tmp = tmp.drop_duplicates()
loc_code_only_list = list(tmp[0])
loc_code_only_list

['11680',
 '11740',
 '11305',
 '11500',
 '11620',
 '11215',
 '11530',
 '11545',
 '11350',
 '11320',
 '11230',
 '11590',
 '11440',
 '11410',
 '11650',
 '11200',
 '11290',
 '11710',
 '11470',
 '11560',
 '11170',
 '11380',
 '11110',
 '11140',
 '11260']

### Trade crawling init

In [81]:
def crawling_init(input_area_code, input_date, service_key):
    main_url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD='+input_area_code+'&DEAL_YMD='+str(input_date)+'&serviceKey='+service_key+''
    response = requests.get(url=main_url)

    result = json.dumps(xmltodict.parse(response.text), ensure_ascii=False)
    
    try:
        tmp_dict = json.loads(result)
        result_dict = tmp_dict["response"]["body"]["items"]["item"]
        print(result_dict)
        
        print(' ')
        print('Current code: '+ input_area_code)
        print('Current year & month: '+ input_date)
        print('Deal Count: '+ str(len(result_dict)))
        print(' ')

        return result_dict

    except Exception as ex:  # 에러 종류
        print('에러가 발생 했습니다', ex)
        ### ex는 발생한 에러의 이름을 받아오는 변수
        ### 바로 stackoverflow 로 이동하자!!
#         url = "http://stackoverflow.com/search?q=[python]+" + str(ex)
#         webbrowser.open(url, new=2)  # new=2 means new tab


In [86]:
service_key = 'KoJIN7Wy0N%2F%2FcHVQw1iYgJ1rCYBvTUPGSNQD18L0k6KSRsgf2s5cF4N6bmcJ%2B%2FZD5RUJt56Xz5OkUgox3x%2FW5Q%3D%3D'

In [9]:
crawling_init('11110', '201901', service_key)

[{'거래금액': '38,300', '건축년도': '2003', '년': '2019', '법정동': '명륜1가', '아파트': '송림아마레스아파트', '월': '1', '일': '11~20', '전용면적': '53.42', '지번': '2-12', '지역코드': '11110', '층': '6'}, {'거래금액': '13,500', '건축년도': '2012', '년': '2019', '법정동': '숭인동', '아파트': '삼전솔하임2차', '월': '1', '일': '1~10', '전용면적': '16.67', '지번': '296-19', '지역코드': '11110', '층': '5'}, {'거래금액': '85,000', '건축년도': '2004', '년': '2019', '법정동': '평창동', '아파트': '에지앙빌', '월': '1', '일': '11~20', '전용면적': '110.61', '지번': '179-5', '지역코드': '11110', '층': '2'}]
 
Current code: 11110
Current year & month: 201901
Deal Count: 3
 


[{'거래금액': '38,300',
  '건축년도': '2003',
  '년': '2019',
  '법정동': '명륜1가',
  '아파트': '송림아마레스아파트',
  '월': '1',
  '일': '11~20',
  '전용면적': '53.42',
  '지번': '2-12',
  '지역코드': '11110',
  '층': '6'},
 {'거래금액': '13,500',
  '건축년도': '2012',
  '년': '2019',
  '법정동': '숭인동',
  '아파트': '삼전솔하임2차',
  '월': '1',
  '일': '1~10',
  '전용면적': '16.67',
  '지번': '296-19',
  '지역코드': '11110',
  '층': '5'},
 {'거래금액': '85,000',
  '건축년도': '2004',
  '년': '2019',
  '법정동': '평창동',
  '아파트': '에지앙빌',
  '월': '1',
  '일': '11~20',
  '전용면적': '110.61',
  '지번': '179-5',
  '지역코드': '11110',
  '층': '2'}]

### mkdir & crawl

In [99]:
for y in range(2018, 2019):
    year_path = sub_path + "/" + str(y)
    for m in range(10, 13):
        month_path = year_path+"/"+str(m).zfill(2)
        for code in loc_code_only_list:
            file_path = month_path+'/'+code+'.txt'
            sum_date = str(y)+""+str(m).zfill(2)
            if not os.path.exists(os.path.dirname(file_path)):
                try:
                    os.makedirs(os.path.dirname(file_path))
                except OSError as exc:
                    if exc.errno != errno.EEXIST:
                        raise
                        
            result = crawling_init(code, sum_date, service_key)
            with open(file_path, 'wb') as f:
                pickle.dump(result, f)

[{'거래금액': '105,000', '건축년도': '2007', '년': '2018', '법정동': '역삼동', '아파트': '쌍용플레티넘밸류', '월': '10', '일': '1~10', '전용면적': '84.9', '지번': '826-37', '지역코드': '11680', '층': '12'}, {'거래금액': '20,017', '건축년도': '2012', '년': '2018', '법정동': '역삼동', '아파트': '요진와이시티미니', '월': '10', '일': '1~10', '전용면적': '12.1', '지번': '727-9', '지역코드': '11680', '층': '7'}, {'거래금액': '158,000', '건축년도': '2007', '년': '2018', '법정동': '역삼동', '아파트': '롯데캐슬노블', '월': '10', '일': '1~10', '전용면적': '154.26', '지번': '835-68', '지역코드': '11680', '층': '7'}, {'거래금액': '32,000', '건축년도': '2003', '년': '2018', '법정동': '역삼동', '아파트': '한양럭스', '월': '10', '일': '1~10', '전용면적': '59.78', '지번': '628-1', '지역코드': '11680', '층': '4'}, {'거래금액': '74,000', '건축년도': '2003', '년': '2018', '법정동': '역삼동', '아파트': '예지향', '월': '10', '일': '1~10', '전용면적': '84.6', '지번': '728-52', '지역코드': '11680', '층': '4'}, {'거래금액': '60,000', '건축년도': '2002', '년': '2018', '법정동': '역삼동', '아파트': '명인갤러리', '월': '10', '일': '1~10', '전용면적': '68.37', '지번': '729-37', '지역코드': '11680', '층': '7'}, {'거래금액': '20,000',

[{'거래금액': '44,200', '건축년도': '2010', '년': '2018', '법정동': '명일동', '아파트': '명일동 다성이즈빌아파트', '월': '10', '일': '1~10', '전용면적': '67.54', '지번': '358', '지역코드': '11740', '층': '3'}, {'거래금액': '88,000', '건축년도': '1985', '년': '2018', '법정동': '명일동', '아파트': '고덕주공9', '월': '10', '일': '1~10', '전용면적': '83.93', '지번': '257', '지역코드': '11740', '층': '7'}, {'거래금액': '62,000', '건축년도': '1983', '년': '2018', '법정동': '명일동', '아파트': '삼익그린2차', '월': '10', '일': '1~10', '전용면적': '42.93', '지번': '15', '지역코드': '11740', '층': '7'}, {'거래금액': '62,800', '건축년도': '1983', '년': '2018', '법정동': '명일동', '아파트': '삼익그린2차', '월': '10', '일': '1~10', '전용면적': '42.93', '지번': '15', '지역코드': '11740', '층': '3'}, {'거래금액': '82,500', '건축년도': '1983', '년': '2018', '법정동': '명일동', '아파트': '삼익그린2차', '월': '10', '일': '1~10', '전용면적': '66.87', '지번': '15', '지역코드': '11740', '층': '9'}, {'거래금액': '60,500', '건축년도': '1983', '년': '2018', '법정동': '명일동', '아파트': '삼익그린2차', '월': '10', '일': '1~10', '전용면적': '42.93', '지번': '15', '지역코드': '11740', '층': '2'}, {'거래금액': '62,900', '건축년도': '1984

[{'거래금액': '81,000', '건축년도': '2011', '년': '2018', '법정동': '미아동', '아파트': '두산위브트레지움', '월': '10', '일': '1~10', '전용면적': '114.88', '지번': '811', '지역코드': '11305', '층': '6'}, {'거래금액': '59,950', '건축년도': '2010', '년': '2018', '법정동': '미아동', '아파트': '삼성래미안트리베라2단지', '월': '10', '일': '1~10', '전용면적': '58.99', '지번': '812', '지역코드': '11305', '층': '2'}, {'거래금액': '43,500', '건축년도': '2004', '년': '2018', '법정동': '미아동', '아파트': '에스케이북한산시티', '월': '10', '일': '1~10', '전용면적': '84.86', '지번': '1353', '지역코드': '11305', '층': '3'}, {'거래금액': '75,000', '건축년도': '2017', '년': '2018', '법정동': '미아동', '아파트': '꿈의숲롯데캐슬', '월': '10', '일': '1~10', '전용면적': '84.98', '지번': '4', '지역코드': '11305', '층': '2'}, {'거래금액': '60,000', '건축년도': '2003', '년': '2018', '법정동': '미아동', '아파트': '경남아너스빌(1356-0)', '월': '10', '일': '1~10', '전용면적': '114.922', '지번': '1356', '지역코드': '11305', '층': '2'}, {'거래금액': '49,000', '건축년도': '2006', '년': '2018', '법정동': '미아동', '아파트': '미아1차래미안', '월': '10', '일': '1~10', '전용면적': '59.9', '지번': '797', '지역코드': '11305', '층': '8'}, {'거래금액': '

[{'거래금액': '79,500', '건축년도': '2008', '년': '2018', '법정동': '염창동', '아파트': '염창1차보람더하임', '월': '10', '일': '1~10', '전용면적': '84.98', '지번': '305', '지역코드': '11500', '층': '8'}, {'거래금액': '56,250', '건축년도': '1998', '년': '2018', '법정동': '염창동', '아파트': '동아', '월': '10', '일': '1~10', '전용면적': '59.97', '지번': '292', '지역코드': '11500', '층': '11'}, {'거래금액': '41,800', '건축년도': '2007', '년': '2018', '법정동': '염창동', '아파트': '노블레스2(267-4)', '월': '10', '일': '1~10', '전용면적': '83.29', '지번': '267-4', '지역코드': '11500', '층': '9'}, {'거래금액': '58,000', '건축년도': '1999', '년': '2018', '법정동': '염창동', '아파트': '무학', '월': '10', '일': '1~10', '전용면적': '59.38', '지번': '255', '지역코드': '11500', '층': '17'}, {'거래금액': '68,000', '건축년도': '1995', '년': '2018', '법정동': '염창동', '아파트': '삼성한마음', '월': '10', '일': '1~10', '전용면적': '84.87', '지번': '244-2', '지역코드': '11500', '층': '6'}, {'거래금액': '75,000', '건축년도': '1998', '년': '2018', '법정동': '염창동', '아파트': '한강동아', '월': '10', '일': '1~10', '전용면적': '114.68', '지번': '293', '지역코드': '11500', '층': '15'}, {'거래금액': '50,000', '건축년도': 

[{'거래금액': '42,000', '건축년도': '1998', '년': '2018', '법정동': '봉천동', '아파트': '무학', '월': '10', '일': '1~10', '전용면적': '81.95', '지번': '1531-39', '지역코드': '11620', '층': '3'}, {'거래금액': '72,400', '건축년도': '2014', '년': '2018', '법정동': '봉천동', '아파트': '관악파크푸르지오', '월': '10', '일': '1~10', '전용면적': '74.87', '지번': '1728', '지역코드': '11620', '층': '1'}, {'거래금액': '37,000', '건축년도': '2007', '년': '2018', '법정동': '봉천동', '아파트': '에메랄드102동', '월': '10', '일': '1~10', '전용면적': '39.6', '지번': '41-259', '지역코드': '11620', '층': '5'}, {'거래금액': '75,000', '건축년도': '2003', '년': '2018', '법정동': '봉천동', '아파트': '관악드림(동아)', '월': '10', '일': '1~10', '전용면적': '114.75', '지번': '1712', '지역코드': '11620', '층': '9'}, {'거래금액': '66,500', '건축년도': '1996', '년': '2018', '법정동': '봉천동', '아파트': '보라매삼성', '월': '10', '일': '1~10', '전용면적': '84.84', '지번': '1698-1', '지역코드': '11620', '층': '7'}, {'거래금액': '62,500', '건축년도': '2014', '년': '2018', '법정동': '봉천동', '아파트': '관악파크푸르지오', '월': '10', '일': '1~10', '전용면적': '59.98', '지번': '1728', '지역코드': '11620', '층': '8'}, {'거래금액': '62,500'

...

[{'거래금액': '116,000', '건축년도': '2003', '년': '2018', '법정동': '방배동', '아파트': '방배대우멤버스카운티', '월': '12', '일': '1~10', '전용면적': '101.59', '지번': '809-22', '지역코드': '11650', '층': '4'}, {'거래금액': '95,000', '건축년도': '2001', '년': '2018', '법정동': '방배동', '아파트': '방배팰리스빌라', '월': '12', '일': '11~20', '전용면적': '118.09', '지번': '809-14', '지역코드': '11650', '층': '2'}, {'거래금액': '150,000', '건축년도': '1981', '년': '2018', '법정동': '방배동', '아파트': '신동아', '월': '12', '일': '11~20', '전용면적': '105.86', '지번': '988-1', '지역코드': '11650', '층': '4'}, {'거래금액': '124,000', '건축년도': '2008', '년': '2018', '법정동': '방배동', '아파트': '방배어울림', '월': '12', '일': '21~31', '전용면적': '122.785', '지번': '998', '지역코드': '11650', '층': '5'}, {'거래금액': '55,000', '건축년도': '2003', '년': '2018', '법정동': '방배동', '아파트': '푸른마을', '월': '12', '일': '21~31', '전용면적': '108.45', '지번': '763-12', '지역코드': '11650', '층': '6'}, {'거래금액': '140,000', '건축년도': '2013', '년': '2018', '법정동': '우면동', '아파트': '서초네이처힐3단지', '월': '12', '일': '1~10', '전용면적': '114.79', '지번': '717', '지역코드': '11650', '층': '3'}, {'거래금

[{'거래금액': '20,200', '건축년도': '2015', '년': '2018', '법정동': '성북동1가', '아파트': '휴아름 스튜디오', '월': '12', '일': '21~31', '전용면적': '16.53', '지번': '35-22', '지역코드': '11290', '층': '14'}, {'거래금액': '15,700', '건축년도': '2015', '년': '2018', '법정동': '성북동1가', '아파트': '휴아름 스튜디오', '월': '12', '일': '21~31', '전용면적': '13.57', '지번': '35-22', '지역코드': '11290', '층': '10'}, {'거래금액': '41,000', '건축년도': '1998', '년': '2018', '법정동': '돈암동', '아파트': '한진(609-1)', '월': '12', '일': '1~10', '전용면적': '59.58', '지번': '609-1', '지역코드': '11290', '층': '17'}, {'거래금액': '41,500', '건축년도': '1998', '년': '2018', '법정동': '돈암동', '아파트': '한진(609-1)', '월': '12', '일': '1~10', '전용면적': '59.58', '지번': '609-1', '지역코드': '11290', '층': '16'}, {'거래금액': '67,000', '건축년도': '1998', '년': '2018', '법정동': '돈암동', '아파트': '한신', '월': '12', '일': '11~20', '전용면적': '132.96', '지번': '609-1', '지역코드': '11290', '층': '10'}, {'거래금액': '46,000', '건축년도': '1998', '년': '2018', '법정동': '돈암동', '아파트': '한신', '월': '12', '일': '11~20', '전용면적': '68.13', '지번': '609-1', '지역코드': '11290', '층': '9'}, {'거래금

[{'거래금액': '152,000', '건축년도': '2007', '년': '2018', '법정동': '잠실동', '아파트': '트리지움', '월': '12', '일': '1~10', '전용면적': '84.83', '지번': '35', '지역코드': '11710', '층': '11'}, {'거래금액': '169,700', '건축년도': '2006', '년': '2018', '법정동': '잠실동', '아파트': '레이크팰리스', '월': '12', '일': '1~10', '전용면적': '116.19', '지번': '44', '지역코드': '11710', '층': '12'}, {'거래금액': '174,750', '건축년도': '1978', '년': '2018', '법정동': '잠실동', '아파트': '주공아파트 5단지', '월': '12', '일': '1~10', '전용면적': '76.5', '지번': '27', '지역코드': '11710', '층': '11'}, {'거래금액': '174,000', '건축년도': '1978', '년': '2018', '법정동': '잠실동', '아파트': '주공아파트 5단지', '월': '12', '일': '11~20', '전용면적': '76.5', '지번': '27', '지역코드': '11710', '층': '11'}, {'거래금액': '135,000', '건축년도': '2008', '년': '2018', '법정동': '잠실동', '아파트': '리센츠', '월': '12', '일': '11~20', '전용면적': '84.99', '지번': '22', '지역코드': '11710', '층': '10'}, {'거래금액': '165,000', '건축년도': '2008', '년': '2018', '법정동': '잠실동', '아파트': '리센츠', '월': '12', '일': '11~20', '전용면적': '84.99', '지번': '22', '지역코드': '11710', '층': '13'}, {'거래금액': '125,000', '건축년도':

[{'거래금액': '12,500', '건축년도': '2012', '년': '2018', '법정동': '영등포동1가', '아파트': '(145-0)', '월': '12', '일': '1~10', '전용면적': '14.24', '지번': '145', '지역코드': '11560', '층': '5'}, {'거래금액': '10,900', '건축년도': '2011', '년': '2018', '법정동': '영등포동1가', '아파트': '헤렌어반', '월': '12', '일': '21~31', '전용면적': '13.331', '지번': '144', '지역코드': '11560', '층': '3'}, {'거래금액': '12,500', '건축년도': '2014', '년': '2018', '법정동': '영등포동8가', '아파트': '리안펠리체', '월': '12', '일': '11~20', '전용면적': '15.71', '지번': '45-8', '지역코드': '11560', '층': '8'}, {'거래금액': '129,000', '건축년도': '1975', '년': '2018', '법정동': '여의도동', '아파트': '한양', '월': '12', '일': '21~31', '전용면적': '109.42', '지번': '42', '지역코드': '11560', '층': '8'}, {'거래금액': '63,000', '건축년도': '1997', '년': '2018', '법정동': '당산동1가', '아파트': '진로', '월': '12', '일': '1~10', '전용면적': '59.4', '지번': '16-1', '지역코드': '11560', '층': '3'}, {'거래금액': '54,500', '건축년도': '2014', '년': '2018', '법정동': '당산동3가', '아파트': '당산계룡리슈빌2단지', '월': '12', '일': '1~10', '전용면적': '38.77', '지번': '566', '지역코드': '11560', '층': '18'}, {'거래금액': '57,500',

[{'거래금액': '43,900', '건축년도': '2000', '년': '2018', '법정동': '면목동', '아파트': '두산5', '월': '12', '일': '1~10', '전용면적': '60', '지번': '1522-1', '지역코드': '11260', '층': '15'}, {'거래금액': '48,500', '건축년도': '2006', '년': '2018', '법정동': '면목동', '아파트': '경남아너스빌', '월': '12', '일': '11~20', '전용면적': '59.858', '지번': '1529', '지역코드': '11260', '층': '10'}, {'거래금액': '11,000', '건축년도': '2013', '년': '2018', '법정동': '면목동', '아파트': '현대휴온', '월': '12', '일': '11~20', '전용면적': '18.17', '지번': '1535', '지역코드': '11260', '층': '3'}, {'거래금액': '12,600', '건축년도': '2013', '년': '2018', '법정동': '면목동', '아파트': '현대휴온', '월': '12', '일': '11~20', '전용면적': '19.83', '지번': '1535', '지역코드': '11260', '층': '7'}, {'거래금액': '8,450', '건축년도': '2013', '년': '2018', '법정동': '면목동', '아파트': '신영', '월': '12', '일': '21~31', '전용면적': '14.43', '지번': '538', '지역코드': '11260', '층': '8'}, {'거래금액': '30,800', '건축년도': '1988', '년': '2018', '법정동': '면목동', '아파트': '한신', '월': '12', '일': '21~31', '전용면적': '44.5', '지번': '193-1', '지역코드': '11260', '층': '15'}, {'거래금액': '57,250', '건축년도': '2005', '

### get old Data

In [98]:
with open('../KB/Dataset/Price/deal_whole_data.txt', 'rb') as f:
    load_data = pickle.load(f)
old_data = pd.DataFrame(load_data)

In [97]:
old_data

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"29,500",1983,2006,당주동,세종,1,11~20,93.4,100,11110,6
1,"15,741",1998,2006,익선동,운현신화타워,1,1~10,57.18,30-6,11110,12
2,"23,198",1998,2006,익선동,운현신화타워,1,1~10,84.26,30-6,11110,6
3,"23,198",1998,2006,익선동,운현신화타워,1,11~20,84.26,30-6,11110,14
4,"21,535",1998,2006,익선동,운현신화타워,1,11~20,78.23,30-6,11110,15
5,"21,500",1999,2006,명륜2가,아남3,1,11~20,47.43,237,11110,6
6,"20,000",1999,2006,명륜2가,아남3,1,11~20,44.37,237,11110,8
7,"16,800",1992,2006,창신동,창신쌍용1,1,11~20,54.7,702,11110,8
8,"16,500",1993,2006,창신동,창신쌍용2,1,11~20,64.66,703,11110,11
9,"28,000",1993,2006,창신동,창신쌍용2,1,11~20,106.62,703,11110,7


### dir loop & get recent Data

In [100]:
whole_data = []
for y in range(2018, 2020):
    year_path = sub_path + "/" + str(y)
    for m in range(1, 13):
        month_path = year_path+"/"+str(m).zfill(2)
        for code in loc_code_only_list:
            file_path = month_path+'/'+code+'.txt'
            sum_date = str(y)+""+str(m).zfill(2)
            
            print(sum_date, file_path)
            if not os.path.exists(os.path.dirname(file_path)):
                print('dir not exist')
            else:
                with open(file_path, 'rb') as f:
                    data = pickle.load(f)
#                     print('==='*40)
                    if data is None:
                        print('Oh, shit')
                    else:
                        print('Yes, it works')
                        for item in data:
                            whole_data.append(item)
                
print("="*30)
print("whole_count: "+ str(len(whole_data)))
print("-"*30)

# final_data_path = sub_path+'/deal_whole_data2.txt'
# if not os.path.exists(os.path.dirname(final_data_path)):
#     try:
#         os.makedirs(os.path.dirname(final_data_path))
#     except OSError as exc:
#         if exc.errno != errno.EEXIST:
#             raise

# result = whole_data
# with open(final_data_path, 'wb') as f:
#     pickle.dump(result, f)

201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11680.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11740.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11305.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11500.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11620.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11215.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11530.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11545.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11350.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11320.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/01/11230.txt
dir not exist
201801 /Users/tax_k/Projects/AI-DL/KB-v2/Da

201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11545.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11350.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11320.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11230.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11590.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11440.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11410.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11650.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11200.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11290.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2018/11/11710.txt
Yes, it works
201811 /Users/tax_k/Projects/AI-DL/KB-v2/Da

Oh, shit
201903 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/03/11110.txt
Oh, shit
201903 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/03/11140.txt
Oh, shit
201903 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/03/11260.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11680.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11740.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11305.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11500.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11620.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11215.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11530.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11545.txt
Oh, shit
201904 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/04/11350.txt
Oh, shit
201904 

Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11530.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11545.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11350.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11320.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11230.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11590.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11440.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11410.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11650.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11200.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11290.txt
Oh, shit
201912 /Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/2019/12/11710.txt
Oh, shit
201912 

In [130]:
add_df = pd.DataFrame(whole_data[:6137])

In [139]:
add_df2 = pd.DataFrame(whole_data[6148:])

In [140]:
add_df2

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"50,000",1999,2019,등촌동,제일(코오롱),1,1~10,59.95,642,11500,19
1,"42,900",1995,2019,등촌동,"등촌1,2단지주공아파트",1,1~10,41.85,687,11500,6
2,"25,700",2002,2019,화곡동,계명아파트다동,1,1~10,56.88,343-45,11500,5
3,"32,000",2004,2019,화곡동,유림노르웨이숲(919-1),1,1~10,76.23,919-1,11500,2
4,"10,000",2012,2019,화곡동,강서동도센트리움,1,11~20,14.677,1161,11500,5
5,"25,800",1999,2019,화곡동,삼도,1,11~20,70.84,913-1,11500,9
6,"40,000",1992,2019,가양동,강변,1,11~20,39.6,1475,11500,3
7,"85,000",2005,2019,내발산동,우장산힐스테이트,1,1~10,59.98,657,11500,16
8,"50,000",1994,2019,방화동,장미,1,1~10,59.04,841,11500,9
9,"9,450",2013,2019,방화동,Air Palace(619-7),1,11~20,14.1,619-7,11500,4


In [141]:
frames = [add_df, add_df2]
conc_result = pd.concat(frames)

In [142]:
conc_result

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"105,000",2007,2018,역삼동,쌍용플레티넘밸류,10,1~10,84.9,826-37,11680,12
1,"20,017",2012,2018,역삼동,요진와이시티미니,10,1~10,12.1,727-9,11680,7
2,"158,000",2007,2018,역삼동,롯데캐슬노블,10,1~10,154.26,835-68,11680,7
3,"32,000",2003,2018,역삼동,한양럭스,10,1~10,59.78,628-1,11680,4
4,"74,000",2003,2018,역삼동,예지향,10,1~10,84.6,728-52,11680,4
5,"60,000",2002,2018,역삼동,명인갤러리,10,1~10,68.37,729-37,11680,7
6,"20,000",2003,2018,역삼동,우림루미아트1,10,11~20,24,706-11,11680,7
7,"205,000",2010,2018,역삼동,래미안그레이튼(진달래2차),10,11~20,105.89,762-3,11680,8
8,"20,000",2003,2018,역삼동,우림루미아트1,10,11~20,23.66,706-11,11680,7
9,"20,500",2003,2018,역삼동,우림루미아트2,10,11~20,23.9,706-10,11680,4


In [143]:
frames = [old_data, conc_result]
ref_data = pd.concat(frames)

In [145]:
ref_data = ref_data.drop_duplicates()
ref_data

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"29,500",1983,2006,당주동,세종,1,11~20,93.4,100,11110,6
1,"15,741",1998,2006,익선동,운현신화타워,1,1~10,57.18,30-6,11110,12
2,"23,198",1998,2006,익선동,운현신화타워,1,1~10,84.26,30-6,11110,6
3,"23,198",1998,2006,익선동,운현신화타워,1,11~20,84.26,30-6,11110,14
4,"21,535",1998,2006,익선동,운현신화타워,1,11~20,78.23,30-6,11110,15
5,"21,500",1999,2006,명륜2가,아남3,1,11~20,47.43,237,11110,6
6,"20,000",1999,2006,명륜2가,아남3,1,11~20,44.37,237,11110,8
7,"16,800",1992,2006,창신동,창신쌍용1,1,11~20,54.7,702,11110,8
8,"16,500",1993,2006,창신동,창신쌍용2,1,11~20,64.66,703,11110,11
9,"28,000",1993,2006,창신동,창신쌍용2,1,11~20,106.62,703,11110,7


In [147]:
fin_path = sub_path + '/trade_data_201901.txt'
fin_path

'/Users/tax_k/Projects/AI-DL/KB-v2/Dataset/Trade/trade_data_201901.txt'

In [149]:
with open(fin_path, 'wb') as f:
    pickle.dump(ref_data, f)